## Install required libraries:

In [1]:
!pip install opencv-python
!pip install tensorflow
!pip install pandas
!pip install sklearn
!pip install cv2
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 26 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=d3e86a5a8c952e6a259b79f9404cef7adc3ce68790310df45b0b861d4fe9a9a4
  Stored in directory

## Connecting into the google drive in order to get access to the trained models and target video files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive
%cd MyDrive
!ls

/content/drive
/content/drive/MyDrive
 allSols.pdf			     ToIpad
 Archive.zip			     unknown
 AVLTreeNode.java		     Year2.1
'awe-domk-zei – 4 בפבר׳ 2022.gjam'   year3
 COCO				    'year3 (1)'
'Colab Notebooks'		    'אישור מנהל קהילה.pdf'
'cppfiles 22.59.11'		     דיסקרטית
 cppfiles.zip			    'דיסקרטית תרגולים'
 deepLearning			    'הרצאות מודרנית'
 DL_FINAL			    'הרצאות מודרנית (1)'
 doknow				    'הרצאות סמסטר א'
 imagesForHw			    "הרצאות סמסטר א'"
'java tests'			    'משפטים מהמצגות סיבוכיות.pdf'
 mivnat2			    'פתרונות ענבל.pdf'
 Notability			    'שיעורי בית סמסטר א'
 Notability1			    'שיעורי בית סמסטר א (1)'
 Notability2			    'תמונות אייפו לא למחוק!'
 oldPcNiv			    'תרגול 11'
 PATIENT_DATA			    'תרגילי בית'
 People


## Main Program:
### Face detection, Gender Estimation & Age Estimation

In [11]:
import cv2 as cv2
import face_recognition
import numpy as np
import numpy
from tensorflow.keras.models import load_model
model = load_model("./DL_FINAL/GenderEstimation.h5")
model2 = load_model("./DL_FINAL/AgeEstimation.h5")

fvs = cv2.VideoCapture('./DL_FINAL/kidShortVideo.mp4')
faces_count = 0
framecounter = 15
while fvs.isOpened():
    fvs.set(cv2.CAP_PROP_POS_FRAMES, framecounter)
    ret, frame = fvs.read()
    if not ret:
        break
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        face_crop = numpy.copy(frame[top - 25:bottom + 25, left - 25:right + 25])
        # Extract the image for the gender estimation model and fit her to his input size
        imageForGender = cv2.resize(face_crop,(224,224))
        imageForGender = cv2.cvtColor(imageForGender,cv2.COLOR_BGR2RGB)
        imageForGender = imageForGender / 255.
        imageForGender = numpy.expand_dims(imageForGender,axis=0)

        genderEstimation = numpy.argmax(model.predict(imageForGender))
        genderEstimation = numpy.where(genderEstimation==0,"Female","Male")
        # Extract the image for the age estimation model and fir her to his input size
        imageForAge = cv2.resize(face_crop,(224,224))
        imageForAge = imageForAge / 255
        imageForAge = numpy.expand_dims(imageForAge,axis=0)
        ageEstimation = model2.predict(imageForAge)
        # Print the result for the image and save her in the drive
        print(f'For facecount {faces_count} age = {int(ageEstimation)} gender = {genderEstimation}')
        cv2.imwrite("./People/" + "FrameNumber" + str(faces_count) + "_" + str(genderEstimation) + "_" + str(int(ageEstimation)) + ".jpg",
                    frame[top - 25:bottom + 25, left - 25:right + 25])
        
        faces_count += 1
    print(" [INFO] Processing frame", framecounter)
    framecounter += 15
print("\n [INFO] Finish detecting faces")
fvs.release()
cv2.destroyAllWindows()

For facecount 0 age = 4 gender = Male
 [INFO] Processing frame 15
For facecount 1 age = 5 gender = Male
 [INFO] Processing frame 30
For facecount 2 age = 9 gender = Male
 [INFO] Processing frame 45
For facecount 3 age = 4 gender = Male
 [INFO] Processing frame 60

 [INFO] Finish detecting faces
